In [28]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import yake
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [29]:
nlp = spacy.load("pt_core_news_lg")

In [39]:
data = pd.read_csv('spam.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'spam.csv'

In [15]:
# Função para calcular os pesos TF-IDF
def calcular_tfidf(textos):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(textos)
    words = vectorizer.get_feature_names_out()
    tfidf_weights = tfidf_matrix.toarray()
    return words, tfidf_weights

In [16]:
# Função para calcular os pesos YAKE
def calcular_yake(textos):
    kw_extractor = yake.KeywordExtractor(lan="pt")
    yake_weights = []
    for texto in textos:
        keywords = kw_extractor.extract_keywords(texto)
        yake_weights.append({kw: score for kw, score in keywords})
    return yake_weights

In [17]:
# Processar os textos com spaCy para lematização
lemmatized_texts = [" ".join([token.lemma_ for token in nlp(texto)]) for texto in textos]

In [18]:
# Calcular os pesos TF-IDF
words, tfidf_weights = calcular_tfidf(lemmatized_texts)

In [19]:
# Calcular os pesos YAKE
yake_weights = calcular_yake(textos)

In [10]:
# Combinar os pesos TF-IDF e YAKE
combined_weights = []
for i, texto in enumerate(textos):
    combined_score = {}
    for word in words:
        tfidf_score = tfidf_weights[i][words.tolist().index(word)] if word in words else 0
        yake_score = yake_weights[i].get(word, 0)
        combined_score[word] = tfidf_score + (1 - yake_score)  # Combina os pesos (1 - yake_score) porque YAKE scores são inversamente proporcionais
    combined_weights.append(combined_score)

In [11]:
# Imprimir os resultados
for i, texto in enumerate(textos):
    print(f"Texto: {texto}")
    sorted_scores = sorted(combined_weights[i].items(), key=lambda x: x[1], reverse=True)
    for word, score in sorted_scores[:5]:  # Mostrar as top 5 palavras com mais peso
        print(f"Palavra: {word}, Peso Combinado: {score:.4f}")
    print("\n")

Texto: A aprendizagem de máquina é fascinante e está transformando indústrias.
Palavra: estar, Peso Combinado: 1.4133
Palavra: indústria, Peso Combinado: 1.4133
Palavra: transformar, Peso Combinado: 1.4133
Palavra: de, Peso Combinado: 1.2441
Palavra: ser, Peso Combinado: 1.2441


Texto: A inteligência artificial é uma área importante da ciência da computação.
Palavra: de, Peso Combinado: 1.4095
Palavra: área, Peso Combinado: 1.3466
Palavra: um, Peso Combinado: 1.2636
Palavra: computação, Peso Combinado: 1.2510
Palavra: ser, Peso Combinado: 1.2047


Texto: O aprendizado profundo é uma subárea da aprendizagem de máquina.
Palavra: de, Peso Combinado: 1.4641
Palavra: um, Peso Combinado: 1.2988
Palavra: ser, Peso Combinado: 1.2320
Palavra: máquina, Peso Combinado: 1.1405
Palavra: aprendizado, Peso Combinado: 1.0955




In [22]:
X_train, X_test, y_train, y_test = train_test_split(combined_weights, classes, test_size=0.3, random_state=42)

NameError: name 'classes' is not defined